# Wine Dataset

In questo progetto andrò ad utilizzare alcune librerie di python (numpy e pandas per la gestione dei dati , pyplot e seaborn per disegnare i grafici e scikit-learn per gli algoritmi di learning) per risolvere due tasks:
- Partendo in maniera arbitraria da uno dei due dataset, andrò a calcolare la qualità di un vino in base alle sue caratteristiche chimiche.
- Prendendo in considerazione entrambi i dataset andrò a verificare se un vino è bianco o rosso sempre in base alle stesse caratteristiche.

## Primo problema Classificazione dei vini in base alla qualità 

### Import e caricamento dei dati

In [ ]:
# IMPORT

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPClassifier

In [2]:
# CARICAMENTO DEI DATI

# SCELTA DEL DATASET
#nome_dataset = "winequality-white.csv";
nome_dataset = "winequality-red.csv";

dataset = pd.read_csv(nome_dataset, sep=";")
colonne = dataset.columns.tolist()
X = dataset[colonne[:-1]] 
y = dataset[colonne[-1]] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# PER QUESTA LIBRERIA NON GENERO IL CROSS-VALIDATION VISTO CHE AUTOMATIZZA IL TUTTO
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
print("Ho caricato il dataset", nome_dataset)

Ho caricato il dataset winequality-red.csv


### Descrizione e stampa dei dati 

In [ ]:
# DESCRIZIONE DEI DATI 

dataset.describe()

In [ ]:
# STAMPA DISTRIBUZIONE DELLE LABEL

plt.bar(dataset[colonne[-1]].value_counts().sort_index().index, dataset[colonne[-1]].value_counts().sort_index().values)

In [ ]:
# PRIMA STAMPA DEI DATI POSSO VEDERE LA DISTRIBUZIONE DI DUE VARIABILI RISPETTO AD OGNI QUALITA

feature_1 = 6
feature_2 = 5
sns.relplot(
    data=dataset, x=colonne[feature_1], y=colonne[feature_2], col=colonne[-1], height=4, aspect=.7, col_wrap=4
)
plt.show()

In [ ]:
# SECONDA STAMPA IN QUESTO CASO VADO A VEDERE PER UNA FEATURE SCELTA TRAMITE UNA VARIABILE COLONNA_PLOT LA DISTIBUZIONE IN 
# PERCENTUALE DEI NUM_INTERVALLI INTERVALLI UTILIZZATI PER OGNI QUANTITA

# DISCRETIZZO TUTTE LE FEATURES IN MODO DA POTER STAMPARE SEMPLICEMENTE UN STIMA DI COME VA OGNI COLONNA

num_intervalli = 8
colonna_plot = 9


dataplot = dataset.copy()
for colonna in range(len(colonne) - 1):
    minimo = dataset[colonne[colonna]].min()
    massimo = dataset[colonne[colonna]].max()
    passo = (massimo-minimo)/(num_intervalli - 1)
    bins = np.arange(minimo, massimo, passo)
    if(len(bins) == num_intervalli - 1):
        bins = np.append(np.arange(minimo, massimo, passo), massimo)
    nuova_colonna = np.digitize(dataset[colonne[colonna]],bins,right=True) 
    dataplot[colonne[colonna]] = nuova_colonna + 1
    
# GENERO LA MATRICE MAT CONTANDO LE OCCORRENZE DI OGNI VALORE DISCRETO PER OGNI QUANTITÀ POSSIBILE IN USCITA

mat = np.zeros((num_intervalli,10)) 
for j in range(len(dataplot)):
    mat[dataplot[colonne[colonna_plot]][j] -1 , dataplot[colonne[-1]][j] - 1]+=1
    
    
# GENERO LA MATRICE MAT_PERCENTUALI ANDANDO A CALCOLARE UNA STATISTICA SU QUANTO É PRESENTE OGNI VALORE DISCRETO

somma_colonne=sum(mat)
mat_percentuali = mat.copy()
num_righe, num_colonne = mat.shape
for i in range(num_righe):
    for j in range(num_colonne):
        if(somma_colonne[j] == 0):
            mat_percentuali[i, j] = 0
        else:
            mat_percentuali[i, j] = mat[i, j] / somma_colonne[j]

# STAMPA DEI GRAFICI LE QUALITA SONO SOLO DA 1 A 9 VISTO CHE IN QUESTI DATASET AL MASSIMO ABBIAMO QUALITA TRA 3 E 9

names= list(range(1,num_intervalli + 1))
fig, axs = plt.subplots(3, 3)
zoom = 3
w, h = fig.get_size_inches()
fig.set_size_inches(w * zoom, h * zoom)
axs[0, 0].bar(names, mat_percentuali[:, 0])
axs[0, 0].set_title('qualità 1')
axs[0, 1].bar(names, mat_percentuali[:, 1])
axs[0, 1].set_title('qualità 2')
axs[0, 2].bar(names, mat_percentuali[:, 2])
axs[0, 2].set_title('qualità 3')
axs[1, 0].bar(names, mat_percentuali[:, 3])
axs[1, 0].set_title('qualità 4')
axs[1, 1].bar(names, mat_percentuali[:, 4])
axs[1, 1].set_title('qualità 5')
axs[1, 2].bar(names, mat_percentuali[:, 5])
axs[1, 2].set_title('qualità 6')
axs[2, 0].bar(names, mat_percentuali[:, 6])
axs[2, 0].set_title('qualità 7')
axs[2, 1].bar(names, mat_percentuali[:, 7])
axs[2, 1].set_title('qualità 8')
axs[2, 2].bar(names, mat_percentuali[:, 8])
axs[2, 2].set_title('qualità 9')
fig.suptitle('Statistiche per la colonna ' + colonne[colonna_plot], fontsize=16)
plt.show()


In [ ]:
# TERZA STAMPA MATRICE DI CORRELAZIONE

corr = dataset.corr()
fig, ax = plt.subplots(figsize=(12, 10))
fig.suptitle('Matrice di correlazione', fontsize=16)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)
plt.show()
    

### Feature Scaling 

In [3]:
X_train_scal = X_train.copy()
X_test_scal = X_test.copy()
scale = StandardScaler().fit(X_train_scal)
X_train_scal = scale.transform(X_train_scal)
X_test_scal = scale.transform(X_test_scal)

#PER RITRASFORMARLI IN DATAFRAME

X_train_scal = pd.DataFrame(data=X_train_scal, index=X_train.index, columns=[colonne[:-1]])
X_test_scal = pd.DataFrame(data=X_test_scal, index=X_test.index, columns=[colonne[:-1]])

# PER QUESTA LIBRERIA NON GENERO IL CROSS-VALIDATION VISTO CHE AUTOMATIZZA IL TUTTO
#X_val_scal = X_val.copy()
#X_val_scal = scale.transform(X_val_scal)
#X_val_scal = pd.DataFrame(data=X_val_scal, index=X_val.index, columns=[colonne[:-1]])

print("Scaling delle features effettuato")

Scaling delle features effettuato


### Training e calcolo dell'accuratezza dei diversi algoritmi settati a 1 

Effettuo il training per 6 diversi algoritmi

In [7]:
esegui_DT = 0   # DECISION TREE
esegui_SVM = 0  # SVM
esegui_RL = 0   # REGRESSIONE LOGISTICA
esegui_NB = 0   # NAIVE BAYES
esegui_RF = 1   # RANDOM FOREST
esegui_NN = 0   # RETE NEURALE

def scegli_modello(parametri, algoritmo, X, y):
    modello = GridSearchCV(algoritmo, parametri)
    modello.fit(X, y)
    return modello
  
def calcola_accuratezza_modello(modello, X_test, y_test, nome_algoritmo):
    predizione= modello.predict(X_test)
    print("Accuratezza di " + nome_algoritmo + " = " + str(accuracy_score(y_test, predizione)))
    return predizione

if(esegui_DT == 1):
    parametri_DT = {'criterion':('gini', 'entropy')}
    modello_DT = scegli_modello(parametri_DT, DecisionTreeClassifier(), X_train_scal, y_train)
    best_params_DT = modello_DT.best_params_
    pred_DT = calcola_accuratezza_modello(modello_DT, X_test_scal, y_test, "Decision Tree")
    
if(esegui_SVM == 1):
    parametri_SVM = {'C':np.logspace(-1, 5, 4)}
    modello_SVM = scegli_modello(parametri_SVM, SVC(), X_train_scal, y_train)
    best_params_SVM = modello_SVM.best_params_
    pred_SVM = calcola_accuratezza_modello(modello_SVM, X_test_scal, y_test, "SVM")
    
if(esegui_RL == 1):
    parametri_RL = {'C':np.logspace(9, 10, 3)}
    modello_RL = scegli_modello(parametri_RL, LogisticRegression(max_iter=400), X_train_scal, y_train)
    best_params_RL = modello_RL.best_params_
    pred_RL = calcola_accuratezza_modello(modello_RL, X_test_scal, y_test, "Regressione Logistica")

if(esegui_NB == 1):
    parametri_NB = {'var_smoothing': np.logspace(0,-9, num=300)}
    modello_NB = scegli_modello(parametri_NB, GaussianNB(), X_train_scal, y_train)
    best_params_NB = modello_NB.best_params_
    pred_NB = calcola_accuratezza_modello(modello_NB, X_test_scal, y_test, "Naive Bayes")

if(esegui_RF == 1):
    parametri_RF = {'max_depth': [10, 20, 30], 'min_samples_leaf':[3 ,10, 30, 100]}
    modello_RF = scegli_modello(parametri_RF, RandomForestClassifier(n_estimators=1000), X_train_scal, y_train)
    best_params_RF = modello_RF.best_params_
    pred_RF = calcola_accuratezza_modello(modello_RF, X_test_scal, y_test, "Random Forest")
    
if(esegui_NN == 1):
    parametri_NN = {'alpha':np.logspace(-3, 1, 5)}
    modello_NN = scegli_modello(parametri_NN, MLPClassifier(hidden_layer_sizes=(1000, 500, 300, 30, 10), max_iter=600), X_train_scal, y_train)
    best_params_NN = modello_NN.best_params_
    pred_NN = calcola_accuratezza_modello(modello_NN, X_test_scal, y_test, "NN")

Accuratezza di Random Forest = 0.825


### Stampa delle curve di learning dei diversi algoritmi settati a 1

In [ ]:
esegui_DT = 0   # DECISION TREE
esegui_SVM = 0  # SVM
esegui_RL = 0   # REGRESSIONE LOGISTICA
esegui_NB = 0   # NAIVE BAYES
esegui_RF = 1   # RANDOM FOREST
esegui_NN = 0   # RETE NEURALE

def plot_learning_curves(X, y, modello, nome_algoritmo):
    train_sizes, train_scores, validation_scores = learning_curve(estimator = modello, X = X, y = y)
    validation_mean = np.mean(validation_scores, axis=1)
    train_mean = np.mean(train_scores, axis=1)
    plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training")
    plt.plot(train_sizes, validation_mean, color="#111111", label="Cross-validation")
    plt.title("Curve di learning per " + nome_algoritmo)
    plt.xlabel("Dimensione Training Set"), plt.ylabel("Accuratezza"), plt.legend(loc="best")
    plt.tight_layout()
    plt.show()
    
if(esegui_DT == 1 and 'modello_DT' in locals()):
    plot_learning_curves(X_train_scal, y_train, DecisionTreeClassifier(**best_params_DT), 'Decision Tree')
    
if(esegui_SVM == 1 and 'modello_SVM' in locals()):
    plot_learning_curves(X_train_scal, y_train, SVC(**best_params_SVM), 'SVM')    

if(esegui_RL == 1 and 'modello_RL' in locals()):
    plot_learning_curves(X_train_scal, y_train, LogisticRegression(**best_params_RL, max_iter=100), 'Regressione Logistica')    

if(esegui_NB == 1 and 'modello_NB' in locals()):
    plot_learning_curves(X_train_scal, y_train, GaussianNB(**best_params_NB), 'Naive Bayes')

if(esegui_RF == 1 and 'modello_RF' in locals()):
    plot_learning_curves(X_train_scal, y_train, RandomForestClassifier(**best_params_RF, n_estimators=1000), 'Random Forest')

if(esegui_RF == 1 and 'modello_NN' in locals()):
    plot_learning_curves(X_train_scal, y_train, MLPClassifier(hidden_layer_sizes=(1000, 500, 300), max_iter=600, **best_params_NN), 'Rete Neurale')

### Calcolo dell statistiche per i diversi algoritmi settati a 1 

In [11]:
esegui_DT = 1   # DECISION TREE
esegui_SVM = 1  # SVM
esegui_RL = 1   # REGRESSIONE LOGISTICA
esegui_NB = 1   # NAIVE BAYES
esegui_RF = 1   # RANDOM FOREST
esegui_NN = 1   # RETE NEURALE

if(esegui_DT == 1 and 'pred_DT' in locals()):
    print("\nConfusion Matrix per il Decision Tree\n")
    print(str(confusion_matrix(y_test, pred_DT)))
    print("\nPrecision e Recall per il Decision Tree\n")
    print(classification_report(y_test, pred_DT))
    
if(esegui_SVM == 1 and 'pred_SVM' in locals()):
    print("\nConfusion Matrix per SVM\n")
    print(str(confusion_matrix(y_test, pred_SVM)))
    print("\nPrecision e Recall per SVM\n")
    print(classification_report(y_test, pred_SVM))
    
if(esegui_RL == 1 and 'pred_RL' in locals()):
    print("\nConfusion Matrix per la Regressione Logistica\n")
    print(str(confusion_matrix(y_test, pred_RL)))
    print("\nPrecision e Recall per la Regressione Logistica\n")
    print(classification_report(y_test, pred_RL))

if(esegui_NB == 1 and 'pred_NB' in locals()):
    print("\nConfusion Matrix per il Naive Bayes\n")
    print(str(confusion_matrix(y_test, pred_NB)))
    print("\nPrecision e Recall per il Naive Bayes\n")
    print(classification_report(y_test, pred_NB))

if(esegui_RF == 1 and 'pred_RF' in locals()):
    print("\nConfusion Matrix per il Random Forest\n")
    print(str(confusion_matrix(y_test, pred_RF)))
    print("\nPrecision e Recall per il Random Forest\n")
    print(classification_report(y_test, pred_RF))
    
if(esegui_NN == 1 and 'pred_NN' in locals()):
    print("\nConfusion Matrix per la Rete Neurale\n")
    print(str(confusion_matrix(y_test, pred_NN)))
    print("\nPrecision e Recall per la rete Neurale\n")
    print(classification_report(y_test, pred_NN))


Confusion Matrix per il Random Forest

[[986   2]
 [  5 307]]

Precision e Recall per il Random Forest

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       988
           1       0.99      0.98      0.99       312

    accuracy                           0.99      1300
   macro avg       0.99      0.99      0.99      1300
weighted avg       0.99      0.99      0.99      1300



### Primo accorpamento delle label 

Essendoci pochi vini alle fascie estreme rispetto ai vini di fascia media vado ad accorpare i vini di qualità 3 con i vini di qualità 4 e quelli di qualità 8 e 9 con quelli di qualità 7. Rieseguo poi il learning, il plot e calcolo le statistiche e osservo che le prestazioni migliorano di poco. Inizio però ad individuare l'algoritmo con prestazioni migliori su cui concentrarmi: Il Random Forest.

In [4]:
y_test = y_test.replace(3, 4)
y_test = y_test.replace(9, 7)
y_test = y_test.replace(8, 7)

y_train = y_train.replace(3, 4)
y_train = y_train.replace(9, 7)
y_train = y_train.replace(8, 7)

### Secondo accorpamento delle label

Osservo che ho bisogno di accorpare ancora per miglorare le prestazioni: Accorpo allora i vini di qualità 4 (a cui avevo aggiunto precedentemente quelli di qualità 3) con i vini di qualità 5 in modo da ottenere tre classi di vini: bassa media ed alta qualità.

In [5]:
y_test = y_test.replace(4, 5)
y_test = y_test.replace(5, 0)
y_test = y_test.replace(6, 1)
y_test = y_test.replace(7, 2)

y_train = y_train.replace(4, 5)
y_train = y_train.replace(5, 0)
y_train = y_train.replace(6, 1)
y_train = y_train.replace(7, 2)

# IN QUESTO MODO OTTENGO UN'ACCURATEZZA DEL 70% CON IL RANDOM FOREST

Accorpando in questo modo ottengo un'accuratezza del 71% per i vini bianchi e del 74% per i vini rossi. In questo modo l'accuratezza non è molto alta ma l'f-measure delle tre diverse classi è bilanciato. Nel caso avessimo accorpato i vini di qualità 5 con quelli di qualità 6 invece che con quelli di qualità 4, avremmo ottenuto un'accuratezza abbastanza alta, ma solamente perchè la maggior parte dei vini sarebbe ricaduta nei vini di qualità media. In questo modo però, l'algoritmo sarebbe andato a prevedere solo vini di media qualità avendo per le altre due classi un f-measure basso.

### Terzo accorpamento delle label 

Andando ad accorpare ulteriormente le classi unendo le classe dei vini di media qualità con quella dei vini di alta qualità, riduco il problema ad un problema di classificazione binaria. In questo modo con il random forest raggiungo un f-measure del 82%.

In [6]:
y_test = y_test.replace(2, 1)
y_train = y_train.replace(2, 1)

## Secondo problema classificazione di vini rossi e vini bianchi

### Caricamento dei dati e generazione del dataset

In [9]:
bianchi = pd.read_csv("winequality-white.csv", sep=";")
rossi = pd.read_csv("winequality-red.csv", sep=";")
bianchi["colore"]=0;
rossi["colore"]=1; 
dataset = pd.concat([bianchi, rossi], ignore_index = True)
colonne = dataset.columns.tolist()
X = dataset[colonne[:-1]] 
y = dataset[colonne[-1]]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

A questo punto riscrivo solamente il codice per eseguire Il Random Forest per semplicità; In ogni caso potrei rieseguire lo stesso codice di prima anche per questo problema.

In [12]:
X_train_scal = X_train.copy()
X_test_scal = X_test.copy()
scale = StandardScaler().fit(X_train_scal)
X_train_scal = scale.transform(X_train_scal)
X_test_scal = scale.transform(X_test_scal)
X_train_scal = pd.DataFrame(data=X_train_scal, index=X_train.index, columns=[colonne[:-1]])
X_test_scal = pd.DataFrame(data=X_test_scal, index=X_test.index, columns=[colonne[:-1]])

parametri_RF = {'max_depth': [10, 20, 30], 'min_samples_leaf':[3 ,10, 30, 100]}
modello_RF = scegli_modello(parametri_RF, RandomForestClassifier(n_estimators=1000), X_train_scal, y_train)
best_params_RF = modello_RF.best_params_
pred_RF = calcola_accuratezza_modello(modello_RF, X_test_scal, y_test, "Random Forest")


print("\nConfusion Matrix per il Random Forest\n")
print(str(confusion_matrix(y_test, pred_RF)))
print("\nPrecision e Recall per il Random Forest\n")
print(classification_report(y_test, pred_RF))

Accuratezza di Random Forest = 0.9946153846153846

Confusion Matrix per il Random Forest

[[986   2]
 [  5 307]]

Precision e Recall per il Random Forest

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       988
           1       0.99      0.98      0.99       312

    accuracy                           0.99      1300
   macro avg       0.99      0.99      0.99      1300
weighted avg       0.99      0.99      0.99      1300



Per questo problema ho ottenuto delle prestazioni ottime in quanto con il Random Forest raggiungo un'f-measure del 99.5%